# Import Library

In [1]:
import netCDF4 as nc
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr    
import glob
import os
import cartopy.crs as ccrs
import matplotlib.colors as colors
import pandas as pd


In [2]:
import warnings
warnings.filterwarnings("ignore")

# AGCD

In [4]:
b = xr.open_dataset("/g/data/w97/sl7808/ehf_AGCD/EHI_heatwaves____daily.nc", decode_times=False)
EHIsig=b.EHIsig

In [5]:
event = xr.open_dataset("/g/data/w97/sl7808/ehf_AGCD/EHF_heatwaves____daily.nc").event
EHIsig['time'] = event['time'] 
event
masked_ehi=EHIsig.where(event>0)

In [18]:
dataout = np.empty((110,len(masked_ehi['lat'].values),len(masked_ehi['lon'].values)),dtype=np.float64)
start_year = 1910
for yy in range(110):
    yr=yy+1910
    start_string = '%s-11-01' %(yr)
    end_string = '%s-03-31' %(yr+1)
    dataout[yy,:,:] = masked_ehi.sel(time=slice(start_string,end_string)).sum(dim='time')

dm_new = xr.Dataset({
        'HWC': xr.DataArray(
                data   = dataout,
                dims   = ['year','lat','lon'],
                coords = {'year': np.arange(1910,2020),'lat': masked_ehi['lat'].values,'lon': masked_ehi['lon'].values},
                )
            })
    
#EHFdaily['time'] = event['time'] 

In [19]:
dm_new.to_netcdf('/g/data/w97/sl7808/ALLOUTPUT/EHF_obs/original/cumheat_obs.nc')


<xarray.Dataset>
Dimensions:  (lat: 29, lon: 24, year: 110)
Coordinates:
  * year     (year) int64 1910 1911 1912 1913 1914 ... 2015 2016 2017 2018 2019
  * lat      (lat) float64 -45.0 -43.75 -42.5 -41.25 ... -12.5 -11.25 -10.0
  * lon      (lon) float64 110.6 112.5 114.4 116.2 ... 148.1 150.0 151.9 153.8
Data variables:
    HWC      (year, lat, lon) float64 0.0 0.0 0.0 0.0 ... 27.03 0.2588 0.0 0.0

# ACCESS  HWC Calculation

In [ ]:
EHI_filenames =[]
EHI_filenames = sorted(glob.glob('/g/data/w97/sl7808/ehf/ehi_day_exp*.nc')) 
event_filenames =[]
event_filenames = sorted(glob.glob('/g/data/w97/sl7808/cumheat/var/event/2_event_*.nc')) 
file_name = []
plt.figure(figsize=(20,6))

n=np.arange(1850, 2021, 10)
fig = plt.figure(figsize=(16,5))
ax = fig.add_subplot()
ax.set_xlabel('cumulative heat(oC)')
ax.set_ylabel('year')
ax.set_title("Mean Cumulative heat per season (sum of EHI) in Australia")
ax.set_xticks(n)

#for each ensemble member
for (ehi,event) in zip (EHI_filenames,event_filenames):
    
    file_name = os.path.basename(ehi)
    split_filename = file_name.split('.')
    molecule_name = split_filename[0]
    split_2=molecule_name.rsplit("_",2)
    expnum=split_2[2]
    print(expnum)
    
    EHIsig = xr.open_dataset(ehi, decode_times=False).EHIsig
    event = xr.open_dataset(event).event
    EHIsig['time'] = event['time'] 
    masked_ehi=EHIsig.where(event>0)
  
    
    dataout = np.empty((164,len(masked_ehi['lat'].values),len(masked_ehi['lon'].values)),dtype=np.float64)
    start_year = 1850
    for yy in range(164):
        yr=yy+1850
        start_string = '%s-11-01' %(yr)
        end_string = '%s-03-31' %(yr+1)
        dataout[yy,:,:] = masked_ehi.where(masked_ehi>0).sel(time=slice(start_string,end_string)).sum(dim='time')

    cumheat = xr.Dataset({
            'HWC': xr.DataArray(
                    data   = dataout,
                    dims   = ['year','lat','lon'],
                    coords = {'year': np.arange(1850,2014),'lat': masked_ehi['lat'].values,'lon': masked_ehi['lon'].values},
                    )
                })
    cumheat.to_netcdf('/g/data/w97/sl7808/cumheat/ehi_cumheat_season2_'+ expnum +'.nc')
    cumheat.HWC.mean(dim='lon').mean(dim='lat').plot()
    print("---")
ax.set_xlabel('cumulative heat(oC)')
ax.set_ylabel('year')
ax.set_title("Mean Cumulative heat (sum of EHI) in Australia")
ax.set_xticks(n)

  

## Combine all 20 ensemble members 

In [ ]:
cum_filenames = []
cum_filenames = sorted(glob.glob("/g/data/w97/sl7808/cumheat/ehi_cumheat_season2_*.nc")) 
file_name = []
f=[]

#for each ensemble member
for c in cum_filenames:
    file_name = os.path.basename(c)
    split_filename = file_name.split('.')
    molecule_name = split_filename[0]
    print(molecule_name)
        
    a = xr.open_dataset(c)
    f.insert(-1,a)

cum20_season=xr.concat(f, 'ensemble')
cum20_season